<a href="https://colab.research.google.com/github/zohrabahundov/CBRF/blob/main/CBRF_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt
from pandas import ExcelWriter
from itertools import cycle
import pandas as pd
import numpy as np
import glob
import os

In [2]:
FILE_NAMES = glob.glob('predicted_fact/*.xlsx')
for FILE_NAME in FILE_NAMES:
  os.remove(FILE_NAME)

In [3]:
FILE_NAMES = glob.glob('fact/*.xlsx')

for FILE_NAME in FILE_NAMES:
  print(FILE_NAME)
  for sheet_name in ['Monthly', 'Quarterly']:
    MAX_PERIOD = 12 if sheet_name == 'Monthly' else 4
    MIN_PERIOD = 3

    try:
      df = pd.read_excel(FILE_NAME, sheet_name=sheet_name)
    except:
      continue
    df.columns = [col.replace('Unnamed: 0', 'date') for col in df.columns]

    df = df.loc[df['date'] != 'Преобразование']

    df = df.set_index('date')
    df_all = df.drop(columns=df.columns)

    for target_col in df.columns:
      df = pd.read_excel(FILE_NAME, sheet_name=sheet_name)
      df.columns = [col.replace('Unnamed: 0', 'date') for col in df.columns]

      df = df.loc[df['date'] != 'Преобразование']

      df = df.set_index('date')

      MAX_ALL_INDEX = max(df.index)

      df_forecast = df.loc[df[target_col] == 'Forecast', [target_col]]
      df = df.loc[df[target_col] != 'Forecast', [target_col]]
      df[target_col] = pd.to_numeric(df[target_col])

      data = df.loc[df[target_col].notna(), target_col]
      data.name = 'fact'
      data = data.reset_index()
      data = pd.concat([data, data['date'].str.split('m', expand=True)], axis=1)
      data.columns = [data.columns[0], data.columns[1], 'year', 'month']

      vals = []
      for p in range(MIN_PERIOD, MAX_PERIOD + 1):
        sd = seasonal_decompose(data['fact'], period=p)
        resid_sum = sum(np.abs(sd.resid.fillna(0)))
        vals.append([p, resid_sum])
        ##print(f'p = {p} resid_sum = {resid_sum}')
        
      vals = sorted(vals, key=lambda d: d[1])

      period = vals[0][0]
      ##print(f'period = {period}')

      MIN_VAL = min(data['fact'])
      MAX_VAL = max(data['fact'])
      ##print(f'MIN_VAL = {MIN_VAL} MAX_VAL = {MAX_VAL}')

      if period == MIN_PERIOD:
        #sd = seasonal_decompose(data['fact'], period=1)
        sd = seasonal_decompose(data['fact'], period=period)
      else:
        sd = seasonal_decompose(data['fact'], period=period)

      data['s'] = sd.seasonal
      data['t'] = sd.trend
      data['r'] = sd.resid
      # data['r_abs'] = sd.resid.abs()

      # data['s'].plot()
      # plt.show()

      # data['t'].plot()
      # plt.show()

      # data['r'].plot()
      # plt.show()

      # data['r_abs'].plot()
      # plt.show()

      # data = data.merge(data['fact'][::period], left_index=True, right_index=True, how='outer', suffixes=('', '_y'))
      # data.rename(columns={'fact_y': 's_coef'}, inplace=True)
      # data['s_coef'] = data['s_coef'].ffill()
      # best_season = data.sort_values(by='r_abs').iloc[:1]['s_coef'].values[0]
      # data['s_coef'] = best_season / data['s_coef']
      data['s_coef'] = 1

      ##plt.rcParams["figure.figsize"] = (20, 3)

      ###data['t'].plot()
      minima = argrelextrema(data['t'].values, np.less, order=3)[0]
      maxima = argrelextrema(data['t'].values, np.greater, order=3)[0]

      ###for crack in minima:
      ###  plt.vlines(crack, ymin=data['t'].min(), ymax=data['t'].max(), color='r')
      ###for crack in maxima:
      ###  plt.vlines(crack, ymin=data['t'].min(), ymax=data['t'].max())
      ###plt.show()

      n_periods = 2 * MAX_PERIOD

      locals = np.concatenate((np.array([np.min(data['t'].index.values)]), minima, maxima, np.array([np.max(data['t'].index.values)])), axis=-1)
      locals.sort()

      IS_SIN = True if len(locals) > 0.1 * len(data['fact']) else False

      for i in range(len(locals) - 1):
        arr = data.loc[locals[i]: locals[i+1], 't']
        s = data.loc[locals[i]: locals[i+1], 's']
        s_coef = data.loc[locals[i]: locals[i+1], 's_coef']

        X = arr[data['t'].notna()].index.values.reshape(-1, 1)
        y = arr[data['t'].notna()].values

        lr = LinearRegression()
        lr.fit(X, y)
        ##print(f'from {locals[i]} to {locals[i+1]} y = {lr.coef_[0]}*x + {lr.intercept_}')

        if period > MIN_PERIOD:
          df_pred_ = pd.Series(lr.coef_[0]*arr.index + lr.intercept_ + s*s_coef)
        else:
          df_pred_ = pd.Series(lr.coef_[0]*arr.index + lr.intercept_)
          
        df_pred_.index = arr.index
        data.loc[df_pred_.index, 'predict'] = df_pred_

      max_index = np.max(data['t'].index)
      new_index = np.arange(max_index + 1, max_index + n_periods + 1)
      for _ in range(n_periods):
        data = pd.concat([data, pd.DataFrame([[np.nan] * data.shape[1]], columns=data.columns)], ignore_index=True)

      if period > MIN_PERIOD:
        sezon_start = max_index % MAX_PERIOD
        s0 = data['s'][max_index - sezon_start - period: max_index-sezon_start]
        sx = s0[:]
        for _ in range(5):
          sx = pd.concat([sx, s0], ignore_index=True)
        sx = sx.loc[:len(new_index)-1]
        s0 = sx[:]
        s0.index = new_index
        df_pred_ = pd.Series(lr.coef_[0]*new_index + lr.intercept_ + s0)
      else:
        df_pred_ = pd.Series(lr.coef_[0]*new_index + lr.intercept_)

      df_pred_.index = new_index
      data.loc[df_pred_.index, 'predict'] = df_pred_

      if IS_SIN:
        data.loc[data['predict'] > MAX_VAL, 'predict'] = 2*MAX_VAL - data.loc[data['predict'] > MAX_VAL, 'predict']
        data.loc[data['predict'] < MIN_VAL, 'predict'] = 2*MIN_VAL - data.loc[data['predict'] < MIN_VAL, 'predict']

      data['error'] = data['fact'] - data['predict']

      ###plt.rcParams["figure.figsize"] = (20, 3)
      ###plt.plot(data.index, data['fact'], color='g', label='fact')
      ###plt.plot(data.index, data['predict'], color='r', label='forecast', linestyle = 'dashed')
      ###plt.show()

      months = cycle(data.loc[:11, 'month'].values)
      data['month'] = [next(months) for _ in range(len(data))]

      fy = data['year'].iloc[0]
      years = []

      for shift in range((max(data.index) // MAX_PERIOD) + 1):
        for _ in range(MAX_PERIOD):
          years.append(str(int(fy) + shift).zfill(2))

      data['year'] = years[:len(data['year'])]

      data['date'] = data['year'] + 'm' + data['month']

      data = data[['date', 'fact', 'predict']]
      data = data.set_index('date')
      data = data.loc[:MAX_ALL_INDEX]
      data[target_col] = data['fact'].combine_first(data['predict'])

      df_all = df_all.merge(data[target_col], left_index=True, right_index=True, how='outer')
      
    df_all.index.name=''
      
    if sheet_name == 'Monthly':
      df1 = df_all.loc[:]
      del df_all
    elif sheet_name == 'Quarterly':
      df2 = df_all.loc[:]
      del df_all
    
  #df_all.to_excel(, sheet_name=sheet_name)
  #os.remove(f'predicted_{FILE_NAME}')
  with ExcelWriter(f'predicted_{FILE_NAME}') as writer:
    try:
      df1.to_excel(writer, 'Monthly')
      del df1
    except:
      pass
    try:
      df2.to_excel(writer, 'Quarterly')
      del df2
    except:
      pass

fact/Test_example6.xlsx
fact/Test_example2.xlsx
fact/Test_example14.xlsx
fact/Test_example10.xlsx
fact/Test_example11.xlsx


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


fact/Test_example7.xlsx
fact/Test_example4.xlsx
fact/Train.xlsx
fact/Test_example8.xlsx
fact/Test_example12.xlsx
fact/Test_example9.xlsx
fact/Test_example13.xlsx
fact/Test_example3.xlsx
fact/Test_example1.xlsx
fact/Test_example5.xlsx
